In [1]:
"""
Automated Stock Screener with Telegram Bot Integration
Run this in Google Colab daily to get stock recommendations
"""

# Install required packages
!pip install yfinance pandas numpy matplotlib seaborn requests python-telegram-bot schedule
!pip install alpha_vantage finnhub-python pandas-ta

import yfinance as yf
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration - UPDATE THESE WITH YOUR KEYS
TELEGRAM_BOT_TOKEN = "7604672791:AAFauy6Nakx1hTMgbdbGFuqRhqEtPHcMyiw"  # Get from @BotFather
TELEGRAM_CHAT_ID = "6970413519"     # Get from @userinfobot
ALPHA_VANTAGE_KEY = "8AYVH3X3BTUT20CR"     # Free from alphavantage.co
FINNHUB_KEY = "d1dhtnpr01qn1ojnonggd1dhtnpr01qn1ojnonh0"      # Free from finnhub.io

# Stock universe to screen (you can expand this)
STOCK_UNIVERSE = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'AMD', 'CRM', 'NFLX',
    'ADBE', 'PYPL', 'INTC', 'CSCO', 'ORCL', 'IBM', 'QCOM', 'TXN', 'AVGO', 'MU',
    'SHOP', 'SQ', 'ROKU', 'ZM', 'DOCU', 'SNOW', 'PLTR', 'COIN', 'RBLX', 'U',
    'CRWD', 'ZS', 'OKTA', 'NET', 'DDOG', 'MDB', 'ESTC', 'SPLK', 'NOW', 'WDAY',
    'JPM', 'BAC', 'WFC', 'GS', 'MS', 'C', 'BLK', 'AXP', 'V', 'MA',
    'JNJ', 'PFE', 'UNH', 'ABBV', 'TMO', 'ABT', 'LLY', 'BMY', 'MRK', 'GILD'
]

class StockScreener:
    def __init__(self):
        self.results = []
        self.today = datetime.now().strftime('%Y-%m-%d')

    def send_telegram_message(self, message):
        """Send message via Telegram bot"""
        try:
            url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
            data = {
                'chat_id': TELEGRAM_CHAT_ID,
                'text': message,
                'parse_mode': 'HTML'
            }
            response = requests.post(url, data=data)
            return response.status_code == 200
        except Exception as e:
            print(f"Telegram error: {e}")
            return False

    def get_stock_data(self, symbol, period='1y'):
        """Fetch stock data using yfinance"""
        try:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            info = stock.info
            return hist, info
        except Exception as e:
            print(f"Error fetching {symbol}: {e}")
            return None, None

    def get_financial_data(self, symbol):
        """Get fundamental data"""
        try:
            stock = yf.Ticker(symbol)

            # Financial statements
            financials = stock.financials
            balance_sheet = stock.balance_sheet
            cash_flow = stock.cashflow

            # Key metrics
            info = stock.info

            return {
                'revenue_growth': self.calculate_revenue_growth(financials),
                'margin_trend': self.calculate_margin_trend(financials),
                'fcf_positive': self.check_fcf_positive(cash_flow),
                'debt_equity': info.get('debtToEquity', 0),
                'forward_pe': info.get('forwardPE', 0),
                'trailing_pe': info.get('trailingPE', 0),
                'peg_ratio': info.get('pegRatio', 0),
                'market_cap': info.get('marketCap', 0),
                'current_price': info.get('currentPrice', 0)
            }
        except Exception as e:
            print(f"Financial data error for {symbol}: {e}")
            return None

    def calculate_revenue_growth(self, financials):
        """Calculate YoY revenue growth"""
        try:
            if financials is None or financials.empty:
                return 0

            revenue_row = None
            for idx in financials.index:
                if 'total revenue' in idx.lower() or 'revenue' in idx.lower():
                    revenue_row = financials.loc[idx]
                    break

            if revenue_row is None or len(revenue_row) < 2:
                return 0

            current = revenue_row.iloc[0]
            previous = revenue_row.iloc[1]

            if previous != 0:
                growth = ((current - previous) / abs(previous)) * 100
                return growth
            return 0
        except:
            return 0

    def calculate_margin_trend(self, financials):
        """Check if margins are expanding"""
        try:
            if financials is None or financials.empty:
                return False

            # Look for gross profit or operating income
            margin_row = None
            for idx in financials.index:
                if 'gross profit' in idx.lower() or 'operating income' in idx.lower():
                    margin_row = financials.loc[idx]
                    break

            if margin_row is None or len(margin_row) < 2:
                return False

            current_margin = margin_row.iloc[0]
            previous_margin = margin_row.iloc[1]

            return current_margin > previous_margin
        except:
            return False

    def check_fcf_positive(self, cash_flow):
        """Check if free cash flow is positive"""
        try:
            if cash_flow is None or cash_flow.empty:
                return False

            fcf_row = None
            for idx in cash_flow.index:
                if 'free cash flow' in idx.lower():
                    fcf_row = cash_flow.loc[idx]
                    break

            if fcf_row is None:
                return False

            return fcf_row.iloc[0] > 0
        except:
            return False

    def get_technical_indicators(self, hist_data):
        """Calculate technical indicators"""
        try:
            if hist_data is None or hist_data.empty:
                return {}

            # Moving averages
            hist_data['MA_20'] = hist_data['Close'].rolling(20).mean()
            hist_data['MA_50'] = hist_data['Close'].rolling(50).mean()

            # RSI
            delta = hist_data['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))

            # Volume analysis
            avg_volume = hist_data['Volume'].rolling(20).mean()
            current_volume = hist_data['Volume'].iloc[-1]
            volume_ratio = current_volume / avg_volume.iloc[-1] if avg_volume.iloc[-1] > 0 else 1

            current_price = hist_data['Close'].iloc[-1]
            ma_20 = hist_data['MA_20'].iloc[-1]
            ma_50 = hist_data['MA_50'].iloc[-1]
            current_rsi = rsi.iloc[-1]

            return {
                'current_price': current_price,
                'above_ma20': current_price > ma_20,
                'above_ma50': current_price > ma_50,
                'rsi': current_rsi,
                'volume_ratio': volume_ratio,
                'price_change_5d': ((current_price - hist_data['Close'].iloc[-6]) / hist_data['Close'].iloc[-6]) * 100 if len(hist_data) > 5 else 0
            }
        except Exception as e:
            print(f"Technical analysis error: {e}")
            return {}

    def tier1_screening(self, symbol, financial_data, technical_data):
        """Apply Tier 1 fundamental screening"""
        signals = []

        if not financial_data or not technical_data:
            return signals

        # Financial Performance
        if financial_data.get('revenue_growth', 0) > 15:
            signals.append(f"Revenue growth: {financial_data['revenue_growth']:.1f}% YoY")

        if financial_data.get('margin_trend', False):
            signals.append("Expanding margins")

        if financial_data.get('fcf_positive', False):
            signals.append("Positive free cash flow")

        # Earnings Momentum
        forward_pe = financial_data.get('forward_pe', 0)
        trailing_pe = financial_data.get('trailing_pe', 0)
        if forward_pe > 0 and trailing_pe > 0 and forward_pe < trailing_pe * 0.8:
            signals.append(f"Growth acceleration (Forward P/E: {forward_pe:.1f} vs Trailing: {trailing_pe:.1f})")

        return signals

    def tier2_screening(self, symbol, technical_data):
        """Apply Tier 2 supporting analysis"""
        catalysts = []

        if not technical_data:
            return catalysts

        # Technical momentum
        if technical_data.get('price_change_5d', 0) > 5:
            catalysts.append(f"Strong 5-day momentum: +{technical_data['price_change_5d']:.1f}%")

        if technical_data.get('volume_ratio', 1) > 1.5:
            catalysts.append(f"High volume: {technical_data['volume_ratio']:.1f}x average")

        return catalysts

    def tier3_valuation(self, financial_data, technical_data):
        """Apply Tier 3 valuation check"""
        if not financial_data or not technical_data:
            return False, "Insufficient data"

        checks = []

        # Valuation
        peg = financial_data.get('peg_ratio', 0)
        if 0 < peg < 1.5:
            checks.append(f"Attractive PEG: {peg:.2f}")

        # Technical health
        if technical_data.get('above_ma20', False) and technical_data.get('above_ma50', False):
            checks.append("Above key moving averages")

        rsi = technical_data.get('rsi', 0)
        if 35 <= rsi <= 70:
            checks.append(f"Healthy RSI: {rsi:.1f}")

        return len(checks) >= 1, checks

    def calculate_targets(self, current_price, financial_data):
        """Calculate price targets"""
        # Swing trade targets (1-5 days)
        st_target1 = current_price * 1.05  # 5% swing target
        st_target2 = current_price * 1.10  # 10% swing target

        # Position trade targets (6-18 months)
        forward_pe = financial_data.get('forward_pe', 15)
        if forward_pe > 0:
            # Assume 20% premium for growth
            target1 = current_price * 1.25  # 25% target
            target2 = current_price * 1.50  # 50% target
        else:
            target1 = current_price * 1.20
            target2 = current_price * 1.40

        stop_loss = current_price * 0.85  # 15% stop loss

        return {
            'st_target1': st_target1,
            'st_target2': st_target2,
            'target1': target1,
            'target2': target2,
            'stop_loss': stop_loss
        }

    def screen_stock(self, symbol):
        """Screen individual stock"""
        print(f"Screening {symbol}...")

        # Get data
        hist_data, info = self.get_stock_data(symbol)
        if hist_data is None:
            return None

        financial_data = self.get_financial_data(symbol)
        technical_data = self.get_technical_indicators(hist_data)

        # Apply screening
        tier1_signals = self.tier1_screening(symbol, financial_data, technical_data)
        tier2_catalysts = self.tier2_screening(symbol, technical_data)
        tier3_passed, tier3_checks = self.tier3_valuation(financial_data, technical_data)

        # Check if stock passes minimum criteria
        if len(tier1_signals) >= 2 and len(tier2_catalysts) >= 1 and tier3_passed:
            current_price = technical_data.get('current_price', 0)
            targets = self.calculate_targets(current_price, financial_data)

            return {
                'symbol': symbol,
                'current_price': current_price,
                'market_cap': financial_data.get('market_cap', 0) if financial_data else 0,
                'tier1_signals': tier1_signals,
                'tier2_catalysts': tier2_catalysts,
                'tier3_checks': tier3_checks,
                'targets': targets,
                'forward_pe': financial_data.get('forward_pe', 0) if financial_data else 0,
                'peg_ratio': financial_data.get('peg_ratio', 0) if financial_data else 0
            }

        return None

    def run_daily_screen(self):
        """Run the complete screening process"""
        print(f"Starting daily stock screen for {self.today}")

        passed_stocks = []

        for symbol in STOCK_UNIVERSE:
            try:
                result = self.screen_stock(symbol)
                if result:
                    passed_stocks.append(result)
                    print(f"✅ {symbol} passed screening")
                else:
                    print(f"❌ {symbol} did not pass")
            except Exception as e:
                print(f"Error screening {symbol}: {e}")

        # Sort by market cap (larger companies first)
        passed_stocks.sort(key=lambda x: x['market_cap'], reverse=True)

        # Generate report
        if passed_stocks:
            report = self.generate_report(passed_stocks[:5])  # Top 5 recommendations
            self.send_telegram_message(report)
            print("Report sent to Telegram!")
        else:
            message = f"🔍 Daily Stock Screen - {self.today}\n\nNo stocks passed all screening criteria today. Market conditions may not be favorable for new positions."
            self.send_telegram_message(message)
            print("No stocks passed screening")

        return passed_stocks

    def generate_report(self, stocks):
        """Generate formatted report for Telegram"""
        report = f"📊 <b>Daily Stock Screen Report - {self.today}</b>\n"
        report += f"Found {len(stocks)} high-probability opportunities:\n\n"

        for i, stock in enumerate(stocks, 1):
            symbol = stock['symbol']
            price = stock['current_price']
            market_cap = stock['market_cap'] / 1e9  # Convert to billions

            report += f"<b>{i}. {symbol}</b> - ${price:.2f} (${market_cap:.1f}B)\n"

            # Tier 1 signals
            report += "🎯 <b>Tier 1 Signals:</b>\n"
            for signal in stock['tier1_signals'][:3]:  # Limit to 3 signals
                report += f"  • {signal}\n"

            # Targets
            targets = stock['targets']
            report += f"📈 <b>Targets:</b>\n"
            report += f"  • Swing (1-5d): ${targets['st_target1']:.2f} (+5%) / ${targets['st_target2']:.2f} (+10%)\n"
            report += f"  • Position (6-18m): ${targets['target1']:.2f} (+25%) / ${targets['target2']:.2f} (+50%)\n"
            report += f"  • Stop Loss: ${targets['stop_loss']:.2f} (-15%)\n"

            # Valuation
            if stock['forward_pe'] > 0:
                report += f"💰 Forward P/E: {stock['forward_pe']:.1f}"
                if stock['peg_ratio'] > 0:
                    report += f" | PEG: {stock['peg_ratio']:.2f}"
                report += "\n"

            report += "\n"

        report += "⚠️ <b>Risk Warning:</b> Past performance doesn't guarantee future results. Always do your own research and manage position sizes appropriately.\n\n"
        report += "🤖 Automated analysis - Review fundamentals before trading"

        return report

# Main execution
def main():
    """Main function to run the screener"""

    # Validate configuration
    if (TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE" or
        TELEGRAM_CHAT_ID == "YOUR_CHAT_ID_HERE"):
        print("❌ Please update TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID")
        return

    # Initialize and run screener
    screener = StockScreener()

    # Test Telegram connection
    test_message = f"🤖 Stock Screener Started - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    if screener.send_telegram_message(test_message):
        print("✅ Telegram connection successful")
    else:
        print("❌ Telegram connection failed")
        return

    # Run screening
    results = screener.run_daily_screen()

    print(f"\nScreening complete. Found {len(results)} opportunities.")
    return results

# Run the screener
if __name__ == "__main__":
    results = main()

# Optional: Schedule daily runs (uncomment to use)
"""
import schedule
import time

def job():
    main()

# Schedule daily run at 9:30 AM EST (market open)
schedule.every().day.at("09:30").do(job)

print("Scheduler started. Press Ctrl+C to stop.")
while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute
"""

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=c967553a04e0cb8c2b5c4966d072b453509c476fec4a8aeb058509a8ee0f49eb
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
✅ Telegram connection successful
Starting daily stock screen for 2025-06-24
Screening AAPL...
❌ AAPL did not pass
Screening MSFT...
❌ MSFT did not pass
Screening GOOGL...
❌ GOOGL did not pass
Screening AMZN...
❌ AMZN did not pass
Screening TSLA...
❌ TSLA did not pass
Screening META...
❌ META did not pass
Screening NVDA...
❌ NVDA did not pass
Screening AMD...


ERROR:yfinance:HTTP Error 401: 


✅ AMD passed screening
Screening CRM...
❌ CRM did not pass
Screening NFLX...


ERROR:yfinance:HTTP Error 401: 


❌ NFLX did not pass
Screening ADBE...
❌ ADBE did not pass
Screening PYPL...
❌ PYPL did not pass
Screening INTC...
❌ INTC did not pass
Screening CSCO...
❌ CSCO did not pass
Screening ORCL...
❌ ORCL did not pass
Screening IBM...
❌ IBM did not pass
Screening QCOM...
❌ QCOM did not pass
Screening TXN...


...

ERROR:yfinance:HTTP Error 401: 


❌ ABBV did not pass
Screening TMO...
❌ TMO did not pass
Screening ABT...
❌ ABT did not pass
Screening LLY...
❌ LLY did not pass
Screening BMY...
❌ BMY did not pass
Screening MRK...


ERROR:yfinance:HTTP Error 401: 


❌ MRK did not pass
Screening GILD...
❌ GILD did not pass
Report sent to Telegram!

Screening complete. Found 6 opportunities.


'\nimport schedule\nimport time\n\ndef job():\n    main()\n\n# Schedule daily run at 9:30 AM EST (market open)\nschedule.every().day.at("09:30").do(job)\n\nprint("Scheduler started. Press Ctrl+C to stop.")\nwhile True:\n    schedule.run_pending()\n    time.sleep(60)  # Check every minute\n'